<a href="https://colab.research.google.com/github/naexuis/csScheduler/blob/main/mdh_python_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/naexuis/csScheduler.git

Cloning into 'csScheduler'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 21 (delta 5), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (21/21), 3.25 KiB | 833.00 KiB/s, done.


In [2]:
dir_path = "/content/csScheduler/"

In [3]:
import pandas as pd
import numpy as np
import os
import copy
import json
import matplotlib.pyplot as plt

In [4]:
try:
    import pyomo
except:
    %pip install -q Pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 6.0 MB/s eta 0:00:00


In [5]:
# Import Pyomo Library
import pyomo
from pyomo.environ import *

print(pyomo.__version__)

6.4.4


In [6]:
# Install COIN-OR IPOPT Solver
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

## Problem Description

A new store has been opened which will be open 16 hours a day, 5 days a week. 

Each day, there are **two eight-hour** shifts. 
* Morning shift is from 06:00 to 14:00
* Evening shift is from 14:00 to 22:00

The morning shift will have 5 employees except for Friday when there are 6 employees. 

The evening shift there are 8 employees plus 1 manager except for Friday when there are 10 employees plus 1 manager.

In [7]:
# Define days (1 week)
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Enter shifts of each day
shifts = ['morning', 'evening']  # 2 shifts of 8 hours

In [8]:
# Opening JSON Schedule file to create nested dictionary
with open(dir_path+'Schedule.json') as json_file:
    input_schedule = json.load(json_file)

In [9]:
with open(dir_path+'People.json') as json_file:
    worker_schedule = json.load(json_file)

In [10]:
input_schedule

{'Monday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '2',
    'managerNeeded': 'false'},
   {'shifts': 'evening',
    'numberOfEmployeesNeeded': '2',
    'managerNeeded': 'false'}]},
 'Tuesday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '3',
    'managerNeeded': 'false'},
   {'shifts': 'evening',
    'numberOfEmployeesNeeded': '0',
    'managerNeeded': 'false'}]},
 'Wednesday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '2',
    'managerNeeded': 'false'},
   {'shifts': 'evening',
    'numberOfEmployeesNeeded': '1',
    'managerNeeded': 'false'}]},
 'Thursday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '2',
    'managerNeeded': 'false'},
   {'shifts': 'evening',
    'numberOfEmployeesNeeded': '1',
    'managerNeeded': 'false'}]},
 'Friday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '0',
    'managerNeeded': 'false'}

In [11]:
len(worker_schedule['People'])

10

In [12]:
days_shifts = {day: shifts for day in days}
days_shifts

{'Monday': ['morning', 'evening'],
 'Tuesday': ['morning', 'evening'],
 'Wednesday': ['morning', 'evening'],
 'Thursday': ['morning', 'evening'],
 'Friday': ['morning', 'evening']}

In [13]:
work_shifts = {}

for k in range(len(days)):
    tmpLST = input_schedule[days[k]]['timesWeNeedWorkers']
    tmpWorkLST = []
    for i in range(len(tmpLST)):
        if int(tmpLST[i]['numberOfEmployeesNeeded'])>0:
            tmpWorkLST.append(int(tmpLST[i]['numberOfEmployeesNeeded']))
        else:
            tmpWorkLST.append(0)

    work_shifts[days[k]] = tmpWorkLST

work_shifts

{'Monday': [2, 2],
 'Tuesday': [3, 0],
 'Wednesday': [2, 1],
 'Thursday': [2, 1],
 'Friday': [0, 2]}

In [14]:
# Enter workers ids (name, number, ...)
workers = ['W' + str(i) for i in range(1, 11)]  # 10 workers available, more than needed
workers

['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10']

In [15]:
# We need to create a dictionary of the workers availability where the 
# dictionary key will be a tuple and the value will be a binary variable
# ("worker", "Day", "Shift"): "Available"

c = {}
for k in range(len(worker_schedule['People'])):
    tmpLST = worker_schedule['People'][k]

    for i in range(len(days)):
        tmpLST2 = tmpLST[days[i]]['freeTimes']

        for j in range(len(tmpLST2)):
            c[(tmpLST['name'], days[i], tmpLST2[j]['shifts'])] = int(tmpLST2[j]['available'])

In [16]:
c

{('W1', 'Monday', 'morning'): 0,
 ('W1', 'Monday', 'evening'): 0,
 ('W1', 'Tuesday', 'morning'): 1,
 ('W1', 'Tuesday', 'evening'): 1,
 ('W1', 'Wednesday', 'morning'): 0,
 ('W1', 'Wednesday', 'evening'): 0,
 ('W1', 'Thursday', 'morning'): 0,
 ('W1', 'Thursday', 'evening'): 0,
 ('W1', 'Friday', 'morning'): 1,
 ('W1', 'Friday', 'evening'): 1,
 ('W2', 'Monday', 'morning'): 1,
 ('W2', 'Monday', 'evening'): 1,
 ('W2', 'Tuesday', 'morning'): 1,
 ('W2', 'Tuesday', 'evening'): 1,
 ('W2', 'Wednesday', 'morning'): 0,
 ('W2', 'Wednesday', 'evening'): 0,
 ('W2', 'Thursday', 'morning'): 1,
 ('W2', 'Thursday', 'evening'): 1,
 ('W2', 'Friday', 'morning'): 0,
 ('W2', 'Friday', 'evening'): 0,
 ('W3', 'Monday', 'morning'): 1,
 ('W3', 'Monday', 'evening'): 1,
 ('W3', 'Tuesday', 'morning'): 0,
 ('W3', 'Tuesday', 'evening'): 0,
 ('W3', 'Wednesday', 'morning'): 1,
 ('W3', 'Wednesday', 'evening'): 1,
 ('W3', 'Thursday', 'morning'): 1,
 ('W3', 'Thursday', 'evening'): 1,
 ('W3', 'Friday', 'morning'): 1,
 ('W3',

In [17]:
# We need to create a dictionary of the workers total desired hours

maxhr = {}
for k in range(len(worker_schedule['People'])):
    tmpLST = worker_schedule['People'][k]
    maxhr[tmpLST['name']] = int(tmpLST['hoursDesired'])

In [ ]:
maxhr

Define $W$ as the set of workers, $S$ as the sets of shifts, and $H$ as the set of max number of hours.

Let $c_{ws}$ be worker $j$ availability for shift $s$

Let $x_{ws}$ be the proportion of shift $s$ that is completed by worker $j$.

$${minimize} \sum_{w \in W} \sum_{s \in S} c_{ws}x_{ws}$$

subject to the total numbers of hours $h$ for worker $j$

$$\sum_{s \in S}c_{ws}x_{ws} \le h_{j}$$

the sum of each shift that a worker completes must sum to unity.

$$\sum_{w \in W}x_{ws} = 1$$



In [32]:
M = ConcreteModel()

In [33]:
M.workers = Set(initialize=workers)
M.days = Set(initialize=days)
M.shifts = Set(initialize=shifts)

In [34]:
# Parameters
M.c = Param(M.workers, M.days, M.shifts, initialize=c, default=0)
M.max_hours = Param(M.workers, initialize=maxhr)

In [35]:
M.x = Var(M.workers, M.days, M.shifts, domain=Reals, bounds=(0,1))

In [36]:
M.pprint()

5 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     3 : workers*days*shifts :  100 : {('W1', 'Monday', 'morning'), ('W1', 'Monday', 'evening'), ('W1', 'Tuesday', 'morning'), ('W1', 'Tuesday', 'evening'), ('W1', 'Wednesday', 'morning'), ('W1', 'Wednesday', 'evening'), ('W1', 'Thursday', 'morning'), ('W1', 'Thursday', 'evening'), ('W1', 'Friday', 'morning'), ('W1', 'Friday', 'evening'), ('W2', 'Monday', 'morning'), ('W2', 'Monday', 'evening'), ('W2', 'Tuesday', 'morning'), ('W2', 'Tuesday', 'evening'), ('W2', 'Wednesday', 'morning'), ('W2', 'Wednesday', 'evening'), ('W2', 'Thursday', 'morning'), ('W2', 'Thursday', 'evening'), ('W2', 'Friday', 'morning'), ('W2', 'Friday', 'evening'), ('W3', 'Monday', 'morning'), ('W3', 'Monday', 'evening'), ('W3', 'Tuesday', 'morning'), ('W3', 'Tuesday', 'evening'), ('W3', 'Wednesday', 'morning'), ('W3', 'Wednesday', 'evening'), ('W3', 'Thursday', 'morning'), ('W3